## Imports

In [1]:
from collections import deque
from math import lcm
from tqdm import tqdm

In [2]:
day = "20"

In [3]:
with open(f"input_j{day}.txt") as f:
#with open("test.txt") as f:
    modules = f.readlines()
    modules = list(map(lambda x: x.replace("\n", ""), modules))

nb_modules = len(modules)
nb_modules

58

## Code

In [4]:
part = 2

In [5]:
modules_type = {}
modules_destinations = {}

flip_flops_state = {}
conjunctions_memories = {}

In [6]:
for row in modules[:-1]:

    module_type = row[0]
    module_name = row[1:].split(" ")[0]
    module_destinations = row.split(" -> ")[1].split(", ")

    modules_type[module_name] = module_type
    modules_destinations[module_name] = module_destinations

    if module_type == "%":
        flip_flops_state[module_name] = False
    elif module_type == "&":
        conjunctions_memories[module_name] = {}

modules_type["broadcaster"] = "broadcaster"
modules_destinations["broadcaster"] = modules[-1].split(" -> ")[1].split(", ")

if part == 2:
    modules_type["rx"] = "final"

for module_name in modules_destinations.keys():
    for destination in modules_destinations[module_name]:
        if destination not in modules_type.keys():
             modules_type[destination] = "output"
        if destination in conjunctions_memories.keys():
            conjunctions_memories[destination][module_name] = "low"

## Part 1

In [7]:
nb_pressions = 1000

In [8]:
compteurs = {"high": 0, "low": 0}

for pression in tqdm(range(nb_pressions)):
    a_traiter = deque([("broadcaster", "low", "button")])
    compteurs["low"] += 1
    while len(a_traiter) > 0:
        pulse = a_traiter.popleft()
        type_to_send = pulse[1]
        if modules_type[pulse[0]] == "%":
            if type_to_send == "high":
                type_to_send = False
            else:
                type_to_send = ("low" if flip_flops_state[pulse[0]] else "high")
                flip_flops_state[pulse[0]] = not flip_flops_state[pulse[0]]
        elif modules_type[pulse[0]] == "&":
            conjunctions_memories[pulse[0]][pulse[2]] = pulse[1]
            if all([x == "high" for x in conjunctions_memories[pulse[0]].values()]):
                type_to_send = "low"
            else:
                type_to_send = "high"
        elif modules_type[pulse[0]] in ["output", "final"]:
            type_to_send = False
        if type_to_send:
            for destination in modules_destinations[pulse[0]]:
                a_traiter.append((destination, type_to_send, pulse[0]))
                compteurs[type_to_send] += 1

100%|██████████| 1000/1000 [00:00<00:00, 19609.72it/s]


In [9]:
compteurs

{'high': 40615, 'low': 16772}

In [10]:
compteurs["high"] * compteurs["low"]

681194780

## Part 2

### Brut force (trop long)

In [11]:
compteur_pressions = 0
compteur_low_rx = 0

In [12]:
"""
while compteur_low_rx != 1:
    a_traiter = deque([("broadcaster", "low", "button")])
    compteur_pressions += 1
    compteur_low_rx = 0
    while len(a_traiter) > 0:
        pulse = a_traiter.popleft()
        type_to_send = pulse[1]
        if modules_type[pulse[0]] == "%":
            if type_to_send == "high":
                type_to_send = False
            else:
                type_to_send = ("low" if flip_flops_state[pulse[0]] else "high")
                flip_flops_state[pulse[0]] = not flip_flops_state[pulse[0]]
        elif modules_type[pulse[0]] == "&":
            conjunctions_memories[pulse[0]][pulse[2]] = pulse[1]
            if all([x == "high" for x in conjunctions_memories[pulse[0]].values()]):
                type_to_send = "low"
            else:
                type_to_send = "high"
        elif modules_type[pulse[0]] in ["output", "final"]:
            type_to_send = False
        if type_to_send:
            for destination in modules_destinations[pulse[0]]:
                a_traiter.append((destination, type_to_send, pulse[0]))
                if modules_type[destination] == "final":
                    if type_to_send == "low":
                        compteur_low_rx += 1
                    else:
                        compteur_low_rx += 2
"""

'\nwhile compteur_low_rx != 1:\n    a_traiter = deque([("broadcaster", "low", "button")])\n    compteur_pressions += 1\n    compteur_low_rx = 0\n    while len(a_traiter) > 0:\n        pulse = a_traiter.popleft()\n        type_to_send = pulse[1]\n        if modules_type[pulse[0]] == "%":\n            if type_to_send == "high":\n                type_to_send = False\n            else:\n                type_to_send = ("low" if flip_flops_state[pulse[0]] else "high")\n                flip_flops_state[pulse[0]] = not flip_flops_state[pulse[0]]\n        elif modules_type[pulse[0]] == "&":\n            conjunctions_memories[pulse[0]][pulse[2]] = pulse[1]\n            if all([x == "high" for x in conjunctions_memories[pulse[0]].values()]):\n                type_to_send = "low"\n            else:\n                type_to_send = "high"\n        elif modules_type[pulse[0]] in ["output", "final"]:\n            type_to_send = False\n        if type_to_send:\n            for destination in modules_de

In [13]:
# compteur_pressions

### Méthode plus fine

Dans mon input, l'état final rx n'est atteint que par un seul module de type conjunction appelé &bq. Un signal low ne sera donc envoyé que si chacun des états en mémoire est à "high" dans bq.

In [14]:
conjunctions_memories['bq']

{'vg': 'low', 'kp': 'low', 'gc': 'low', 'tx': 'low'}

4 points d'entrée dans bq, faisons comme au J8 et regardons s'il y a des cycles pour chacun, pour pouvoir faire un PPCM.

In [15]:
compteurs = {}
for module in conjunctions_memories['bq'].keys():
    compteurs[module] = []

compteurs

{'vg': [], 'kp': [], 'gc': [], 'tx': []}

In [16]:
compteur_pressions = 0

In [17]:
while any([len(x) < 5 for x in compteurs.values()]):
    a_traiter = deque([("broadcaster", "low", "button")])
    compteur_pressions += 1
    while len(a_traiter) > 0:
        pulse = a_traiter.popleft()
        type_to_send = pulse[1]
        if modules_type[pulse[0]] == "%":
            if type_to_send == "high":
                type_to_send = False
            else:
                type_to_send = ("low" if flip_flops_state[pulse[0]] else "high")
                flip_flops_state[pulse[0]] = not flip_flops_state[pulse[0]]
        elif modules_type[pulse[0]] == "&":
            conjunctions_memories[pulse[0]][pulse[2]] = pulse[1]
            if pulse[0] == "bq" and pulse[1] == "high":
                compteurs[pulse[2]].append(compteur_pressions)
            if all([x == "high" for x in conjunctions_memories[pulse[0]].values()]):
                type_to_send = "low"
            else:
                type_to_send = "high"
        elif modules_type[pulse[0]] in ["output", "final"]:
            type_to_send = False
        if type_to_send:
            for destination in modules_destinations[pulse[0]]:
                a_traiter.append((destination, type_to_send, pulse[0]))


In [18]:
compteurs

{'vg': [3027, 7054, 11081, 15108, 19135],
 'kp': [2929, 6858, 10787, 14716, 18645],
 'gc': [3001, 7002, 11003, 15004, 19005],
 'tx': [2769, 6538, 10307, 14076, 17845]}

In [19]:
vg = compteurs["vg"][4] - compteurs["vg"][3]
kp = compteurs["kp"][4] - compteurs["kp"][3]
gc = compteurs["gc"][4] - compteurs["gc"][3]
tx = compteurs["tx"][4] - compteurs["tx"][3]

In [20]:
lcm(vg, kp, gc, tx)

238593356738827

Et c'est une bonne réponse !